In [1]:
#!pip install instaloader

In [2]:
import instaloader
from datetime import datetime
from itertools import dropwhile, takewhile
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from string import punctuation

from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')

from nltk.stem import PorterStemmer
porter = PorterStemmer()

from nltk.probability import FreqDist

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zhounanlin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zhounanlin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
L = instaloader.Instaloader()

In [4]:
# not using now due to failure to login

with open('../../ins_login_info.txt', encoding='utf-8') as file:
    content = file.read()
    username = content.split()[0]
    pswd = content.split()[1]

# L.login(username, pswd)
# L.save_session_to_file('session_file')

Default value (read from 'ins_account_usernames.txt') for account_usernames is a list of:
["NBCNews", 
 "FoxNews", 
 "CNN", 
 "NYTimes",
 "WashTimes", 
 "WSJ", 
 "BBCNews", 
 "Reuters",
 "Time", 
 "Aljazeeraenglish", 
 "Tabletmag"]

In [5]:
with open('ins_account_usernames.txt', encoding='utf-8') as file:
    content = file.read()
    ins_usernames_list = content.split()

In [6]:
# ins_usernames_list = ins_usernames_list[10:]
ins_usernames_list

['NBCNews',
 'FoxNews',
 'CNN',
 'NYTimes',
 'WashTimes',
 'WSJ',
 'BBCNews',
 'Reuters',
 'APNews',
 'Time',
 'Aljazeeraenglish',
 'Tabletmag']

For details of properties of Post class:\
https://instaloader.github.io/module/structures.html#posts

For details of properties of Profile class:\
https://instaloader.github.io/module/structures.html?highlight=postcomment#profiles


In [7]:
# for username in account_usernames:
#     df_name = "df_" + username
#     exec_string = "{}.to_csv('latest_ins_csv/{}.csv', index=False)"
#     exec(exec_string.format(df_name, df_name))

**NOTE for this following cell**:\
If successfully scraped for all usernames (no query error popped up), save the exported csv files into a new folder.\
If failed, just skip this cell and run the next cell.

In [8]:
# lists to generate summary info of basic account info
userid_list = []
followers_list = []
full_name_list = []
is_verified_list = []
post_count_list = []    # counts of posts since 231001

for username in ins_usernames_list:
    posts = instaloader.Profile.from_username(L.context, username).get_posts()

    SINCE = datetime(2023, 11, 1)
    UNTIL = datetime.today()
    
    url_code_list = []
    time_utc_list = []
    content_types_list = []
    captions_list = []
    like_counts_list = []
    comment_counts_list = []

    for post in takewhile(lambda p: p.date > SINCE, dropwhile(lambda p: p.date > UNTIL, posts)):
    
        # details of post: date&time, count of likes, count of comments, url
        url_code_list.append(post.shortcode)
        time_utc_list.append(post.date_utc)
        content_types_list.append(post.typename)
        captions_list.append(post.caption)
        like_counts_list.append(post.likes)
        comment_counts_list.append(post.comments)
        # L.download_post(post, username)
    
    # create df with info of each posts from this account
    df_name = "df_" + username
    exec_string = """{} = pd.DataFrame(data={{
    'owner_username': [username] * len(url_code_list),
    'url_code': url_code_list, 
    'time_utc': pd.to_datetime(time_utc_list), 
    'type': content_types_list,
    'caption': captions_list,
    'likes': like_counts_list, 
    'comments': comment_counts_list}})
    """
    exec(exec_string.format(df_name))
    
    # export df to csv
    exec_string = "{}.to_csv('latest_ins_csv/{}.csv', index=False)"
    exec(exec_string.format(df_name, df_name))
    
    print("Done with posts of", username)
    print("{} has {} posts since Nov 1st.".format(username, len(url_code_list)))
    post_count_list.append(len(url_code_list))
    
    
    # append this account info
    userid_list.append(post.owner_profile.userid)
    followers_list.append(post.owner_profile.followers)
    full_name_list.append(post.owner_profile.full_name)
    is_verified_list.append(post.owner_profile.is_verified)
    
d = {'user_id': userid_list, 'followers': followers_list, 'full_name':full_name_list, 'is_verified':is_verified_list, 'posts_since_231001': post_count_list}
df_ins_accounts = pd.DataFrame(data=d)
df_ins_accounts.to_csv('latest_ins_csv/df_ins_accounts.csv', index=False)

Done with posts of NBCNews
NBCNews has 57 posts since October 1st.
Done with posts of FoxNews
FoxNews has 359 posts since October 1st.
Done with posts of CNN
CNN has 95 posts since October 1st.
Done with posts of NYTimes
NYTimes has 111 posts since October 1st.
Done with posts of WashTimes
WashTimes has 94 posts since October 1st.
Done with posts of WSJ
WSJ has 40 posts since October 1st.
Done with posts of BBCNews
BBCNews has 115 posts since October 1st.
Done with posts of Reuters
Reuters has 22 posts since October 1st.
Done with posts of APNews
APNews has 105 posts since October 1st.
Done with posts of Time
Time has 39 posts since October 1st.
Done with posts of Aljazeeraenglish
Aljazeeraenglish has 392 posts since October 1st.
Done with posts of Tabletmag
Tabletmag has 21 posts since October 1st.


In [9]:
stacked_dfs = pd.DataFrame()

try:
    if len(df_ins_accounts) == len(ins_usernames_list):
        print("Successfully scraped for all usernames.")
        print("Remember to save the exported csv files in the directory of './latest_ins_csv' into a new folder")
        
        for username in ins_usernames_list:
            df_name = "df_" + username
            exec("stacked_dfs=pd.concat([stacked_dfs, {}], ignore_index=True)".format(df_name))
            
    else:
        print("Not all usernames were successfully scraped.")
        print("Use backup files instead.")
        
        df_ins_accounts = pd.read_csv('backup_ins_df_231101/df_ins_accounts.csv')
        for username in ins_usernames_list:
            df_name = "df_" + username
            path = 'backup_ins_df_231101/' + df_name + ".csv"
            exec("stacked_dfs=pd.concat([stacked_dfs, pd.read_csv('{}')], ignore_index=True)".format(path))

except:
    print("Failed to scrape.")
    print("Use backup files instead.")
    
    df_ins_accounts = pd.read_csv('backup_ins_df_231101/df_ins_accounts.csv')
    for username in ins_usernames_list:
        df_name = "df_" + username
        path = 'backup_ins_df_231101/' + df_name + ".csv"
        exec("stacked_dfs=pd.concat([stacked_dfs, pd.read_csv('{}')], ignore_index=True)".format(path))

Successfully scraped for all usernames.
Remember to save the exported csv files in the directory of './latest_ins_csv' into a new folder


In [10]:
# from HW2
def find_freq_tokens(tokens, display_limit):
    
    freq_dict = FreqDist(tokens)
    bigram_freq = dict()
    
    sorted_items = sorted(freq_dict.items(), key=lambda item: item[1], reverse=True)
    top_keys = [item[0] for item in sorted_items[:display_limit]]
    
    return top_keys

In [11]:
def text_to_tokens(text):
    
    lower_str = text.lower()
    no_punc_str = ''.join([character for character in lower_str if character not in punctuation])
    no_digit_str = ''.join([character for character in no_punc_str if not character.isdigit()])
    no_special_charac = ''.join([character for character in no_digit_str if character not in ['\u2060', '”', '“']])
    no_dash = no_special_charac.replace('—', ' ').replace('-', ' ').replace('’', ' ')
    orig_tokens = word_tokenize(no_dash)
    
    tokens = [porter.stem(word) for word in orig_tokens if not word in stop_words]
    
    return tokens

In [12]:
stacked_dfs = stacked_dfs.dropna(subset=['caption'])

In [13]:
all_ins_tokens = []

for index, row in stacked_dfs.iterrows():

    tokens = text_to_tokens(row['caption'])
    all_ins_tokens += tokens

top_tokens = find_freq_tokens(all_ins_tokens, 30)
top_tokens

['link',
 'bio',
 'gaza',
 'israel',
 'said',
 'read',
 'israelgazawar',
 'peopl',
 'isra',
 'new',
 'say',
 'palestinian',
 'year',
 'us',
 'one',
 'tap',
 'war',
 'bbcnew',
 'hama',
 '\u2063',
 'time',
 'hospit',
 'kill',
 'state',
 'citi',
 '🔗tap',
 'call',
 'attack',
 '📸',
 'al']

## Pick out the posts that talks about gaza conflict
- method 1: find the posts that contain at least one of the tokens in gaza_tokens_1
- method 2: find the posts that contain at least one of the tokens in gaza_tokens_2, which has two more tokens: "war" and "attack"
- method 3: use a language model to pick

**method 1**

In [14]:
gaza_tokens_1 = ['israel',
 'gaza',
 'isra',
 'hama',
 'palestinian',
 'palestin',
 'israelgazawar',
 'israelhama',
 'israelgaza']


is_about_gaza_1 = []

for index, row in stacked_dfs.iterrows():

    tokens = text_to_tokens(row['caption'])

    found = False
    for g_token in gaza_tokens_1:
        if g_token in tokens:
            found = True
            break

    is_about_gaza_1.append(found)

stacked_dfs['is_about_gaza_1'] = is_about_gaza_1

stacked_dfs[stacked_dfs['is_about_gaza_1']==True]

,owner_username,url_code,time_utc,type,caption,likes,comments,is_about_gaza_1
0,NBCNews,CzrI2-iLQIZ,2023-11-15 16:51:23,GraphSidecar,The families of American citizens kidnapped by...,2486,563,True
1,NBCNews,CzpNZwPuVCA,2023-11-14 22:54:02,GraphVideo,Family members of hostages trapped in Gaza tel...,1008,530,True
2,NBCNews,CzpHbn_rMag,2023-11-14 22:00:26,GraphSidecar,Thousands of demonstrators gathered at the Nat...,13221,2675,True
4,NBCNews,Czo0HjqxDp6,2023-11-14 19:12:17,GraphVideo,@lesterholtnbc sat down with 12 family members...,2154,597,True
13,NBCNews,Czb7Kj0ra6v,2023-11-09 19:03:18,GraphVideo,"Donald Trump, Israel, and a whole slew of insu...",2210,529,True
...,...,...,...,...,...,...,...,...
1443,Tabletmag,CzTxFuSMm95,2023-11-06 15:01:08,GraphSidecar,"From the archives: American universities, thei...",2402,37,True
1445,Tabletmag,CzL6tdaszja,2023-11-03 13:51:17,GraphSidecar,Hezbollah leader Hassan Nasrallah is currently...,1552,54,True
1446,Tabletmag,CzJ9F7pPanZ,2023-11-02 19:33:34,GraphSidecar,"Since the Houthis, Yemen’s Islamist political ...",8233,268,True
1448,Tabletmag,CzHR2zsRRvb,2023-11-01 18:37:17,GraphImage,"How did everyone get Hamas wrong, including th...",10808,391,True


In [15]:
all_ins_GAZA_tokens = []

for index, row in stacked_dfs[stacked_dfs['is_about_gaza_1']==True].iterrows():

    tokens = text_to_tokens(row['caption'])
    all_ins_GAZA_tokens += tokens

len(all_ins_GAZA_tokens)

21139

In [16]:
all_ins_GAZA_tokens = []

for index, row in stacked_dfs[stacked_dfs['is_about_gaza_1']==True].iterrows():

    tokens = word_tokenize(row['caption'])
    all_ins_GAZA_tokens += tokens

len(all_ins_GAZA_tokens)

41988

**method 2**

In [17]:
gaza_tokens_2 = ['israel',
 'gaza',
 'isra',
 'hama',
 'palestinian',
 'attack',
 'palestin',
 'war',
 'israelgazawar',
 'israelhama',
 'israelgaza']


is_about_gaza_2 = []

for index, row in stacked_dfs.iterrows():

    tokens = text_to_tokens(row['caption'])

    found = False
    for g_token in gaza_tokens_2:
        if g_token in tokens:
            found = True
            break

    is_about_gaza_2.append(found)

stacked_dfs['is_about_gaza_2'] = is_about_gaza_2

stacked_dfs[stacked_dfs['is_about_gaza_2']==True]

,owner_username,url_code,time_utc,type,caption,likes,comments,is_about_gaza_1,is_about_gaza_2
0,NBCNews,CzrI2-iLQIZ,2023-11-15 16:51:23,GraphSidecar,The families of American citizens kidnapped by...,2486,563,True,True
1,NBCNews,CzpNZwPuVCA,2023-11-14 22:54:02,GraphVideo,Family members of hostages trapped in Gaza tel...,1008,530,True,True
2,NBCNews,CzpHbn_rMag,2023-11-14 22:00:26,GraphSidecar,Thousands of demonstrators gathered at the Nat...,13221,2675,True,True
4,NBCNews,Czo0HjqxDp6,2023-11-14 19:12:17,GraphVideo,@lesterholtnbc sat down with 12 family members...,2154,597,True,True
10,NBCNews,CzeR47NrZNm,2023-11-10 17:00:10,GraphImage,"Keke Palmer is accusing her ex-boyfriend, Dari...",2988,219,False,True
...,...,...,...,...,...,...,...,...,...
1443,Tabletmag,CzTxFuSMm95,2023-11-06 15:01:08,GraphSidecar,"From the archives: American universities, thei...",2402,37,True,True
1445,Tabletmag,CzL6tdaszja,2023-11-03 13:51:17,GraphSidecar,Hezbollah leader Hassan Nasrallah is currently...,1552,54,True,True
1446,Tabletmag,CzJ9F7pPanZ,2023-11-02 19:33:34,GraphSidecar,"Since the Houthis, Yemen’s Islamist political ...",8233,268,True,True
1448,Tabletmag,CzHR2zsRRvb,2023-11-01 18:37:17,GraphImage,"How did everyone get Hamas wrong, including th...",10808,391,True,True


In [18]:
df_diff = stacked_dfs[stacked_dfs['is_about_gaza_1'] != stacked_dfs['is_about_gaza_2']]
df_diff

,owner_username,url_code,time_utc,type,caption,likes,comments,is_about_gaza_1,is_about_gaza_2
10,NBCNews,CzeR47NrZNm,2023-11-10 17:00:10,GraphImage,"Keke Palmer is accusing her ex-boyfriend, Dari...",2988,219,False,True
17,NBCNews,CzaQg8_Loc1,2023-11-09 03:31:11,GraphImage,Vivek Ramaswamy has been the most attacked can...,2184,143,False,True
42,NBCNews,CzRXKa6r-hH,2023-11-05 16:37:05,GraphVideo,Ukrainian President Volodymyr Zelenskyy tells ...,10122,1196,False,True
155,FoxNews,Czg9kKfM1X9,2023-11-11 18:00:17,GraphImage,ANSWERED THE CALL: A World War II veteran who ...,13940,326,False,True
284,FoxNews,CzTd6gPs5KZ,2023-11-06 12:13:34,GraphImage,DAY IN COURT: The former president has repeate...,7202,1125,False,True
332,FoxNews,CzN3Uu5stFd,2023-11-04 08:00:10,GraphImage,RIDE FROM HELL: The campus is on high alert as...,20842,1575,False,True
389,FoxNews,CzIB1vFN2eD,2023-11-02 01:36:37,GraphImage,LEGAL WOES: A source familiar with the situati...,13700,797,False,True
412,FoxNews,CzFkURUt8tk,2023-11-01 02:40:09,GraphImage,KEEP THE FAITH: The newly elected speaker of t...,19887,1146,False,True
516,NYTimes,Czo-ArJP2tY,2023-11-14 20:39:00,GraphImage,Doctors have long relied on a few key patient ...,9069,188,False,True
530,NYTimes,Czj2lk8uyvr,2023-11-12 20:59:51,GraphVideo,Army Ammunition Plant Is Tied to Mass Shooting...,11467,935,False,True


In [19]:
len(df_diff)

21

In [20]:
for index, row in df_diff.iterrows():

    print(row['caption'])
    print("------------")

Keke Palmer is accusing her ex-boyfriend, Darius Jackson, of physical and emotional abuse, writing in court documents that he has repeatedly attacked her, trespassed at her home and "became very frustrated" with their 8-month-old son. 

Palmer says the abuse happened throughout her two-year relationship with Jackson and did not stop after their breakup in early October.

A judge has granted Palmer a temporary restraining order while she seeks sole custody of their child.

Read more at the link in our bio.
------------
Vivek Ramaswamy has been the most attacked candidate in both of the first two GOP debates.

He was not the target this time around.

NBC News live-tracked every attack, criticism and barb the candidates delivered Wednesday in Miami.

Read more at the link in bio.
------------
Ukrainian President Volodymyr Zelenskyy tells Kristen Welker that Ukraine is “not ready” to give up its “freedom to this f—ing terrorist Putin.” ⁠
⁠
During the exclusive interview with Meet the Press

**Method 3**

## Export the stacked dataframe to csv

In [21]:
stacked_dfs.to_csv('stacked_ins_df.csv', index=False)

Skipping hashtag due to lack of useful posts from the limited and ramdomly fetched posts.

In [22]:
# posts = instaloader.Instaloader().get_hashtag_posts("gaza")
# posts = instaloader.Hashtag.from_name(L.context, "gaza").get_top_posts()

Only top 9 posts of a hashtag can be fetched with the following method

In [23]:
# posts = instaloader.Hashtag.from_name(L.context, "gaza").get_top_posts()